# Module Import

In [2]:
import cv2
import torch
import numpy as np

from PIL import Image
from test_unet.unet import UNet

# 필요한 함수 정의

In [3]:
# 이미지 input을 위한 preprocessing
# UNet 모델의 Preprocessing 절차를 가져옴
def preprocess(img, scale):
    pil_img = Image.fromarray(img)
    trs_w, trs_h = int(1280*scale), int(720*scale)
    pil_img = pil_img.resize((trs_w, trs_h), resample=Image.BICUBIC)
    
    img_ndarray = np.asarray(pil_img)
    img_ndarray = img_ndarray.transpose((2, 0, 1))
    img_ndarray = img_ndarray[np.newaxis, ...]  
    img_ndarray = img_ndarray/255
    
    return torch.as_tensor(img_ndarray.copy())

# 예측 마스킹 데이터를 만들어 내는 메서드
# UNet에서 Wandb로 push하는 로직을 가져옴
def pred_frame(frame, net):
    images = preprocess(frame, .5).to(device=DEVICE, dtype=torch.float32)
    masks_pred = net(images)
    last_im = torch.softmax(masks_pred, dim=1).argmax(dim=1)[0].float().cpu()
    return last_im.numpy()

# 비디오 프레임 데이터와 마스킹 데이터를 merge
'''
------- 원리 ---------
예측한 mask 데이터는 GRAY Scale이기 때문에 값이 0 이 아닌 인덱스를 활용하여
np.ndarray에 인덱싱을 통해 3채널 중 Green으로 변환
'''
def merge_img(frame, pred):
    re_frm = cv2.resize(frame, dsize=(640, 360), interpolation=cv2.INTER_CUBIC)
    add_mask = re_frm.copy()
    add_mask[pred[:,:]!=0]=[0,255,0]
    return add_mask

# 모델 경로 설정

In [4]:
PATH = r"C:\Users\yunjc\_python_jupyter\bupyeonggu\bp_road_crack_detection\1_모델링\unet_result_pth\!dir\checkpoint_epoch5.pth"
MODEL_PATH = PATH.replace("!dir","UNet_b2th10dn50000")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Pre-Trained Parameter 모델

In [5]:
model = UNet(n_channels=3, n_classes=2, bilinear=False)
model.load_state_dict(torch.load(MODEL_PATH))
model.to(device=DEVICE)
model.eval()

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

# CV2로 실행   

In [7]:
width = 640
height = 360
v_num = 7

capture = cv2.VideoCapture(f"D:/data/sample_video{v_num}.mp4")
# capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, width)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

print("재생할 파일 넓이, 높이 : %d, %d"%(width, height))

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter(f'D:/data/UNet_b2th10dn50000/output{v_num}.avi', fourcc, 30.0, (int(width), int(height)))

while cv2.waitKey(33) < 0:
# while capture.isOpened():
    ret, frame = capture.read()
    if not ret:
        print("프레임을 수신할 수 없습니다. 종료 중 ...")
        break

    convert_img = merge_img(frame, pred_frame(frame, model))
    cv2.imshow("Test Vidoe Crack Detect", convert_img)
    out.write(convert_img)


capture.release()
out.release()
cv2.destroyAllWindows()

재생할 파일 넓이, 높이 : 640, 360
